# Welcome To Galapagos Middleware Tutorial

The Galapagos Middleware presents an orchestration framework to create multi-FPGA and CPU networks and can be built on top of any device that has a Galapagos Hypervisor.

## Prerequisites

If you are using the provided container, skip steps 1, 7
0. Install Vivado on your machine
1. `git clone --recursive https://github.com/UofT-HPRC/galapagos.git` <br/> This clones the Galapagos directory
2. `cd <galapagos_dir>`
3. `source build.sh` <br/> This sets up the environment variables needed to run the code within the tutorial
4.  `galapagos-update-board adm-8k5-debug` <br/>
5. `make hlsmiddleware` <br/> This makes all the IPs needed to connect boards.
6. Repeat steps 4, 5 for all other boards you wish to support.
7. `git clone https://github.com/tarafdar/HCAL_HLS4ML` <br/> This is an example project from the HLS4ML project. We will use this kernel within our example


## Galapagos Hypervisor 

The following few cells provide an overview of the Galapagos Hypervisor and Middleware
If you would like to go straight into building your own kernels you can jump to section <a href='#own_example'>Building Your Own Example'</a>


### Hypervisor Overview

The Middleware places Galapagos streaming kernels onto multiple streaming devices. 
A Galapagos streaming devices, includes devices that currently have a Galapagos Hypervisor built for it.
A Galapagos hypervisor abstracts the device as a streaming device.

The following are two shells of two different FPGA Boards we have:


<img style="float: left;" src="fig/7v3_shell.png"/>          <img style="float: left;" src="fig/mpsoc_shell.png"/>

The turqouise part of these hypervisors is where the user application will sit. Note that even though the network and memory interfaces on these boards are different, through the hypervisor the user interacts with both of these devices through the same interfaces.

This principle is also applied to a CPU interface in the following shell:

<img style="float: left;" src="fig/cpu_shell.png"/> 

As you can see across all three shells we have consistent interfaces. This allows us to have portability across different kernels across different boards as well as functional portability between hardware and software.    

## Galapagos Middleware

The middleware places kernels (software and hardware) onto multiple Galapagos devices described in the Galapagos Hypervisor section. A kernel can be visually represented as the following:

<img style="float: left;" src="fig/kernel.png"/>   

Each kernel has a galapagos-stream in and a galapagos-stream out. This is a specific implementation of the AXI-Stream protocol. Each galapagos-stream has a dest and an id field, the dest specifies the target kernel and the id references the source of the packet. Any kernel within the cluster can reference any other kernel by specifiying the kernel destination in the dest field. This destination is independent of the actual physical mapping of the kernel.
This notion can be represented through a logical description and a mapping, with the use of these descriptions we can transform a logical cluster into a physical cluster.

The following is a pictoral example: 

<img style="float: left;" src="fig/logical_to_physical.png"/> 


<a id='own_example'></a>

# Building Your Own Example

The following is an example logical_file:

In [1]:
logical_file = {
	"cluster": {
		"kernel": [
			{
				"num": "0",
				"rep": "1",
				"clk": "ap_clk",
				"aresetn": "ap_rst_n",
				"s_axis": {
					"scope": "global",
					"name": "in_r"
				},
				"m_axis": {
					"scope": "global",
					"name": "out_r"
				},
				"#text": "kern_send"
			},
			{
				"num": "1",
				"rep": "1",
				"clk": "ap_clk",
				"vendor": "xilinx.com",
				"lib": "hls",
				"aresetn": "ap_rst_n",
				"s_axis": {
					"scope": "global",
					"name": "in_r"
				},
				"m_axis": {
					"scope": "global",
					"name": "out_r"
				},
				"id_port": "id_V",
				"#text": "hls4ml_hcal"
			}
		]
	}
}

The above logical file describes two kernels within the cluster. 

The `<num>` field represents the kernel id of that specific kernel. Any other kernel when setting their dest to this number will send a packet to this kernel. <br/>
The `<rep>` field represents the number of times to repeat this kernel within the cluster (on any device). <br/>
The `<clk>` field represents the name of the port that is the clock port. <br/>
The `<aresetn>` field represents the name of the reset n port.

Next each kernel also has an `<s_axis>` which represents the galapagos stream in and the `<m_axis>` which represents the galpagos stream out.
Each of these is set to a `"scope", "global"`. This allows these streams to be directly accessible by any other kernel in the global dest addressing scheme. We also can set extra `"local"` connections, which directly connects kernels together.

The `<id_port>` refers to the name of the port that we wish to attach a constant representing the id of the kernel. This can be used internally to figure out the destinations of other kernels relative to your own id.

The `<#text>` refers to the name of the kernel ip core.

Below is an example of a map_file:

In [2]:
map_file = {
	"cluster": {
		"node": [
			{
				"type": "sw",
				"kernel": [
					"0"
				],
				"mac": "0c:c4:7a:88:c0:47",
				"ip": "10.1.2.155"
			},
			{
				"board": "adm-8k5-debug",
				"comm": "tcp",
				"type": "hw",
				"kernel": "1",
				"mac": "fa:16:3e:55:ca:02",
				"ip": "10.1.2.156"
			}
		]
	}
}

The above map file describes two nodes within the cluster.

The `<type>` field can be hardware or software. <br/>
The `<kernel>` field can be a list or an individual kernel. This refers to the kernels by number in the logical file to be placed in this node. <br/>
The `<mac>` field refers to the nodes L2 Mac address. <br/>
The `<ip>` field refers to the nodes L3 IP Address. <br/>
Currently the only off-chip communication support in software is TCP/IP. However if the node is a hardware node we can specify the `<comm>` field which denotes the off-chip communication protocol to be used. <br/>
Furthermore if the node is a hardware node, the `<board>` denotes the type of board this node is. <br/>

Next we will generate the script files to build our vivado projects using the logical and map file.

In [3]:
import os
from cluster import cluster

project_name = "middleware_test"
path = str(os.environ.get('GALAPAGOS_PATH'))+ "/projects"

cluster_inst = cluster(project_name, logical_file, map_file, mode='dict')
cluster_inst.makeProjectClusterScript(path)
cluster_inst.writeClusterTCL(path, 0)
cluster_inst.writeBRAMFile(path, 'mac')
cluster_inst.writeBRAMFile(path, 'ip')

This will create a cluster in `<galapagos_home>/projects/middleware_test` as we set `middleware_test` as the `project_name`.
However we are not yet ready to create the project as the scripts assume all the IP cores are ready to use.  The section below shows how to create the kernels and where to place them such that our scripts can find all the ip cores.

## Creating User Kernels 

To create a user kernel in hardware you can import the variables with the appropriate part-name by changing the current board by: <br/>
`galapagos-update-board <board_name>`

With the available boardnames as adm-8k5-debug, sidewinder


For this example (if you have vivado hls installed) please: <br/>

```
cd <HLS4ML_HCAL_home>
galapagos-update-board adm-8k5-debug
make hls
```

Make sure all ip cores of kernels are located in `<galapagos_home>/hlsBuild/<board_name>/ip`

This will generate the hls ip for the adm-8k5 board.  The example generate_hls.tcl script ensures the ip cores are correctly placed.

## Building Your Projects

To make your final project navigate to `<galapagos_home>/projects/middleware_test`

`source createCluster.sh`

This will build all the vivado projects.